In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, accuracy_score
import os
from sklearn.svm import SVC
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import load_iris
import lime
import lime.lime_tabular
import shap
import psutil
import time
import os

data = pd.read_csv('KDDTrain+.txt')
columns = (['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted'
,'num_root','num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate'
,'srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate',
'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','attack','level'])

data.columns = columns
data = data.drop('level', axis=1)
data.tail()

# changing attack labels to their respective attack class
def change_label(df):
  df.attack.replace(['apache2','back','land','neptune','mailbomb','pod','processtable','smurf','teardrop','udpstorm','worm'],'Dos',inplace=True)
  df.attack.replace(['ftp_write','guess_passwd','httptunnel','imap','multihop','named','phf','sendmail','snmpgetattack','snmpguess','spy','warezclient','warezmaster','xlock','xsnoop'],'R2L',inplace=True)      
  df.attack.replace(['ipsweep','mscan','nmap','portsweep','saint','satan'],'Probe',inplace=True)
  df.attack.replace(['buffer_overflow','loadmodule','perl','ps','rootkit','sqlattack','xterm'],'U2R',inplace=True)

change_label(data)

le = LabelEncoder()
data['protocol_type']=le.fit_transform(data['protocol_type'])
data['service']=le.fit_transform(data['service'])
data['flag']=le.fit_transform(data['flag'])
data['attack']=le.fit_transform(data['attack'])

def cross_val(x_train, y_train, model):
    accuracies = cross_val_score(estimator = model, X = x_train, y = y_train, cv=5)
    return accuracies.mean()

def fit_and_evaluate(model, x_train , x_test , y_train , y_test):
    model.fit(x_train, y_train)
    
    model_pred = model.predict(x_test)
    model_cross = cross_val(x_train, y_train, model)
    
    return model_cross

def run_experiment(model, x_train , x_test , y_train , y_test):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("R^2 : ", r2_score(y_test, y_pred))
    print("MAE :", mean_absolute_error(y_test,y_pred))
    print("RMSE:",np.sqrt(mean_squared_error(y_test, y_pred)))
    report=classification_report(y_test,y_pred)
    print(report)

x = data.drop(['attack'], axis=1).values
y = data['attack'].values


Modify RandomForest

In [2]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
import tracemalloc
import time
data = pd.read_csv('KDDTrain+.txt')
columns = (['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted'
,'num_root','num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate'
,'srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate',
'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','attack','level'])
data.columns = columns
data = data.drop('level', axis=1)
data.tail()
le = LabelEncoder()
data['protocol_type'] = le.fit_transform(data['protocol_type'])
data['service'] = le.fit_transform(data['service'])
data['flag'] = le.fit_transform(data['flag'])
data['attack'] = le.fit_transform(data['attack'])
x = data.drop(['attack'], axis=1)
y = data['attack']
tracemalloc.start()
start_time = time.time()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(" RandomForestClassifier Accuracy:", accuracy)
current, peak = tracemalloc.get_traced_memory()
print(f"memory usage: {peak / 10**6} MB")
end_time =time.time()
elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time:.4f} seconds")
tracemalloc.stop()

 RandomForestClassifier Accuracy: 0.9984520738241714
memory usage: 94.968884 MB
Elapsed Time: 5.2776 seconds


In [3]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
import tracemalloc
import time
data = pd.read_csv('KDDTrain+.txt')
columns = (['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted'
,'num_root','num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate'
,'srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate',
'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','attack','level'])
data.columns = columns
data = data.drop('level', axis=1)
data.tail()
le = LabelEncoder()
data['protocol_type'] = le.fit_transform(data['protocol_type'])
data['service'] = le.fit_transform(data['service'])
data['flag'] = le.fit_transform(data['flag'])
data['attack'] = le.fit_transform(data['attack'])
x = data.drop(['attack'], axis=1)
y = data['attack']
tracemalloc.start()
start_time = time.time()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=50)
clf = RandomForestClassifier(n_estimators=50, criterion='entropy', max_depth=None, min_samples_split=0.4, min_samples_leaf=1,
                              max_features='log2', oob_score=False, verbose=0, warm_start=False)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("RandomForestClassifier Accuracy:", accuracy)
current, peak = tracemalloc.get_traced_memory()
print(f"memory usage: {peak / 10**6} MB")
end_time =time.time()
elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time:.4f} seconds")
tracemalloc.stop()

RandomForestClassifier Accuracy: 0.8598380609652837
memory usage: 88.758898 MB
Elapsed Time: 0.9333 seconds


Modified

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

mlp = MLPClassifier(hidden_layer_sizes=(150, 100, 50), max_iter=10, alpha=1e-5,
                    solver='adam', verbose=10, tol=1e-4, random_state=42,
                    learning_rate_init=0.001, batch_size=64, early_stopping=True, validation_fraction=0.1)

tracemalloc.start()

start_time = time.time()

mlp.fit(x_train_scaled, y_train)
y_pred_mlp = mlp.predict(x_test_scaled)

accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
print("MLPClassifier Accuracy: ", accuracy_mlp)

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

end_time = time.time()
elapsed_time = end_time - start_time

tracemalloc.stop()

print(f"Elapsed Time: {elapsed_time:.2f} seconds")


Iteration 1, loss = 0.11974618
Validation score: 0.989453
Iteration 2, loss = 0.03499875
Validation score: 0.992062
Iteration 3, loss = 0.03011508
Validation score: 0.990587
Iteration 4, loss = 0.02568939
Validation score: 0.993196
Iteration 5, loss = 0.02359463
Validation score: 0.994783
Iteration 6, loss = 0.02064389
Validation score: 0.993649
Iteration 7, loss = 0.01980911
Validation score: 0.993990
Iteration 8, loss = 0.01972849
Validation score: 0.993763
Iteration 9, loss = 0.01797933
Validation score: 0.994670
Iteration 10, loss = 0.01698797
Validation score: 0.994103
MLPClassifier Accuracy:  0.9936759102455546
Current memory usage is 0.986138MB; Peak was 76.26821MB
Elapsed Time: 82.49 seconds


In [6]:
from xgboost import XGBClassifier

model = XGBClassifier()

tracemalloc.start()

start_time = time.time()

model.fit(x_train, y_train)

y_pred_xgb = model.predict(x_test)

xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
print("XGBClassifier Accuracy: ", xgb_accuracy)

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

end_time = time.time()
elapsed_time = end_time - start_time
tracemalloc.stop()

print(f"Elapsed Time: {elapsed_time:.2f} seconds")


XGBClassifier Accuracy:  0.998756350550381
Current memory usage is 0.402565MB; Peak was 29.692687MB
Elapsed Time: 9.46 seconds


In [7]:
from sklearn.ensemble import AdaBoostClassifier

tracemalloc.start()
start_time = time.time()

abc = AdaBoostClassifier( random_state=0)
abc.fit(x_train, y_train)

y_pred_abc = abc.predict(x_test)

accuracy_abc = accuracy_score(y_test, y_pred_abc)
print("AdaBoostClassifier Accuracy: ", accuracy_abc)

current, peak = tracemalloc.get_traced_memory()
print(f"Memory usage: {peak / 10**6} MB")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

tracemalloc.stop()

AdaBoostClassifier Accuracy:  0.8455493226079593
Memory usage: 81.304239 MB
Time taken: 5.09 seconds


In [9]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier


def ensemble_model(x_train, x_test, y_train, y_test):
    start_time = time.time()
    tracemalloc.start()

    model1 = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=1)
    model2 = MLPClassifier(random_state=1)
    vote_model = VotingClassifier(estimators=[('xgb', model1), ('mlp', model2)], voting='hard')

    vote_model.fit(x_train,y_train)
    y_pred = vote_model.predict(x_test)

    print("accuracy: ", accuracy_score(y_test, y_pred))

    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed Time: {elapsed_time:.4f} seconds")


ensemble_model(x_train, x_test, y_train, y_test)


accuracy:  0.9776936917866215
Current memory usage is 1.25753MB; Peak was 50.841181MB
Elapsed Time: 82.2798 seconds


In [10]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier


def model(x_train, x_test, y_train, y_test):
    start_time = time.time()
    tracemalloc.start()

    model1 = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=1)
    model2 = MLPClassifier(random_state=1)
    vote_model = VotingClassifier(estimators=[('xgb', model1), ('mlp', model2)], voting='hard')

    vote_model.fit(x_train,y_train)
    y_pred = vote_model.predict(x_test)

    print("accuracy: ", accuracy_score(y_test, y_pred))

    current , peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    print(f"Current memory usage is {current / 10**6}MB;")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed Time: {elapsed_time:.4f} seconds")


model(x_train, x_test, y_train, y_test)


accuracy:  0.9776936917866215
Current memory usage is 1.2536MB;
Elapsed Time: 75.1514 seconds


In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

def model(x_train, x_test, y_train, y_test):
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    knn_params = {'n_neighbors': range(3,6), 'weights': ['uniform', 'distance']}
    ada_params = {'n_estimators' : [50,100,150], 'learning_rate' : [0.01, 0.1, 1]}

    knn_grid = GridSearchCV(KNeighborsClassifier(), knn_params, cv=3)
    ada_grid = GridSearchCV(AdaBoostClassifier(random_state=1), ada_params, cv=3)

    model1 = knn_grid.fit(x_train_scaled, y_train)
    model2 = ada_grid.fit(x_train_scaled, y_train)

    vote_model = VotingClassifier(estimators=[('knn_best', model1.best_estimator_), ('ada_best', model2.best_estimator_)], voting='hard')


    tracemalloc.start()
    start_time = time.time()

    vote_model.fit(x_train_scaled, y_train)
    

    y_pred = vote_model.predict(x_test_scaled)
    

    print("accuracy: ", accuracy_score(y_test, y_pred))


    current = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    print(f"Current memory usage is {current[0] / 10**6}MB;")


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed Time: {elapsed_time:.4f} seconds")
    
model(x_train, x_test, y_train, y_test)


accuracy:  0.9090812870448772
Current memory usage is 30.089501MB;
Elapsed Time: 17.1370 seconds


In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC  
from sklearn.naive_bayes import GaussianNB


def model(x_train, x_test, y_train, y_test):
    start_time = time.time()
    tracemalloc.start()

    model1 = KNeighborsClassifier()
    model2 = SVC(random_state=1)  
    model3 = GaussianNB()
    vote_model = VotingClassifier(estimators=[('knn', model1), ('svm', model2), ('gnb', model3)], voting='hard')

    vote_model.fit(x_train, y_train)
    y_pred = vote_model.predict(x_test)

    print("accuracy: ", accuracy_score(y_test, y_pred))

    current , peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    print(f"Current memory usage is {current / 10**6}MB;")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed Time: {elapsed_time:.4f} seconds")


model(x_train, x_test, y_train, y_test)


accuracy:  0.9224703640982218
Current memory usage is 66.722999MB;
Elapsed Time: 826.4001 seconds
